In [1]:
import os
import pandas as pd
import json
import tqdm
import pandas as pd
import urllib.request
import cv2
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import shutil

In [2]:
rev_1 = pd.read_csv('Reviewer_1.csv')
rev_2 = pd.read_csv('Reviewer_2.csv')

In [3]:
rev_1.head()

,id,DataRow ID,Status,External ID,Agreement,Benchmark Agreement,Skipped,Mask_ID
0,cksx1pm147ca10y9i9ewtd3sh,ckseaghtukied0ywvgzdp0ngs,cervical_cancer_positive,A053_1614933917020.png,-1.000000,-1,True,NaN
1,cksx1pnac7duf0y7nevz2bnhf,ckseaghtukiei0ywv8o1506gr,cervical_cancer_positive,A053_1614934171536.png,0.641540,-1,False,cksx20qpx000b266faqsm8afg
2,cksx1u8kq7cct0y9igsi71768,ckseaghtukien0ywvdfxb63ng,cervical_cancer_positive,A053_1614934709068.png,0.658529,-1,False,cksx264ju000h266f2yccx5aj
3,cksx228my7cgp0y9ighq9bude,ckseaghtukies0ywvcdb14nkz,cervical_cancer_positive,A065_1615971362168.png,-1.000000,-1,True,NaN
4,cksx26xu47e6i0y7ncqrp90tx,ckseaghtukiex0ywv3uta7pbn,cervical_cancer_positive,A065_1615971545890.png,-1.000000,-1,True,NaN


In [4]:
rev_2.head()

,id,DataRow ID,Status,External ID,Agreement,Benchmark Agreement,Skipped,Mask_ID
0,cksxews7a7zgn0y64bnho3e4z,ckseaghu0kiso0ywvgkvta9r5,cervical_cancer_positive,M071_1565687060188.png,-1,-1,False,cksxex9ru00003b6ar7bvgcoc
1,cksxewt5f7xdk0y7n117ucnm2,ckseaghu0kist0ywv09qvbg21,cervical_cancer_positive,M075_1565781393764.png,-1,-1,False,cksxf4g4j000e3b6ajua625wu
2,cksxezyio7zlo0y64hbzf3qu4,ckseaghu0kisy0ywvb7k3hs1e,cervical_cancer_positive,M075_1565781481957.png,-1,-1,True,NaN
3,cksxf8gnl7xup0y7nbk691u60,ckseaghu0kit30ywv2bwb5hhz,cervical_cancer_positive,M075_1565781696144.png,-1,-1,False,cksxfbgrq000o3b6agk3ju13o
4,cksxf9ol07zyx0y64bpgfc9wg,ckseaghu0kit80ywv46drgvf9,cervical_cancer_positive,M075_1565781731483.png,-1,-1,False,cksxfdkv3000s3b6a8038ihhp


In [5]:
rev_1.describe(include = ['O'])

,id,DataRow ID,Status,External ID,Mask_ID
count,1082,1082,1082,1082,387
unique,1082,979,1,979,387
top,cksx1pm147ca10y9i9ewtd3sh,ckseaghtukied0ywvgzdp0ngs,cervical_cancer_positive,A053_1614933917020.png,cksx20qpx000b266faqsm8afg
freq,1,2,1082,2,1


In [6]:
rev_1.nunique()

id                     1082
DataRow ID              979
Status                    1
External ID             979
Agreement                35
Benchmark Agreement       1
Skipped                   2
Mask_ID                 387
dtype: int64

# Reviewer 1 remarks
- We see that despite having more images, reviewer 1 had actually only 979 unique images
- The number of labels obtained was only 387


In [7]:
rev_2.describe(include = ['O'])

,id,DataRow ID,Status,External ID,Mask_ID
count,1045,1045,1045,1045,448
unique,1045,1045,1,1045,448
top,cksxews7a7zgn0y64bnho3e4z,ckseaghu0kiso0ywvgkvta9r5,cervical_cancer_positive,M071_1565687060188.png,cksxex9ru00003b6ar7bvgcoc
freq,1,1,1045,1,1


In [8]:
rev_2.nunique()

id                     1045
DataRow ID             1045
Status                    1
External ID            1045
Agreement                 1
Benchmark Agreement       1
Skipped                   2
Mask_ID                 448
dtype: int64

# Reviewer 2 remarks
- Reviewer had all 1045 unique images
- The number of labels obtained was only 448

### **Creating a list of image names**

In [9]:
rev1_imgs = list(rev_1['External ID'])
print(len(rev1_imgs))

1082


In [10]:
rev2_imgs = list(rev_2['External ID'])
print(len(rev2_imgs))

1045


### **Obtaining list of those missing in reviewer_1 csv**

In [11]:
missing = []
for item in rev2_imgs:
    if item not in rev1_imgs:
        missing.append(item)
len(missing)

66

### Reviewer 2 had 66 more unique images than reviewer 1

### Analyzing number of unlabelled images

In [12]:
rev_1['Mask_ID'].isnull().sum()

695

In [13]:
rev_2['Mask_ID'].isnull().sum()

597

In [14]:
rev_1[rev_1['Mask_ID'].isnull()].nunique()

id                     695
DataRow ID             639
Status                   1
External ID            639
Agreement                2
Benchmark Agreement      1
Skipped                  1
Mask_ID                  0
dtype: int64

### Observe that the multiple masks on a single image occur for images whose agreement is not -1

In [15]:
rev_1[(rev_1['Agreement']==-1) & (rev_1['Mask_ID'].notnull() )].nunique()

id                     320
DataRow ID             320
Status                   1
External ID            320
Agreement                1
Benchmark Agreement      1
Skipped                  1
Mask_ID                320
dtype: int64

In [16]:
rev_1[(rev_1['Agreement']!=-1) & (rev_1['Mask_ID'].notnull() )].nunique()

id                     67
DataRow ID             34
Status                  1
External ID            34
Agreement              34
Benchmark Agreement     1
Skipped                 1
Mask_ID                67
dtype: int64

In [17]:
rev_1 = rev_1.drop(['DataRow ID','Status','Agreement', 'Benchmark Agreement','Skipped'],axis=1)
rev_1 = rev_1.dropna()

In [18]:
rev_2 = rev_2.drop(['DataRow ID','Status','Agreement', 'Benchmark Agreement','Skipped'],axis=1)
rev_2 = rev_2.dropna()

In [34]:
imgs = rev_1['External ID']
final_rev_1 = pd.DataFrame()
for item in rev_1['External ID']:
    final = {}
    masks = [None,None,None,None]
    values = rev_1[rev_1['External ID']==item]['Mask_ID']
    IDs = list(values)
    for num in range(len(IDs)):
        masks[num] = IDs[num]
    final['Img_name'] = item
    final['Mask1'] = masks[0]
    final['Mask2'] = masks[1]
    final['Mask3'] = masks[2]
    final['Mask4'] = masks[3]
    data = pd.DataFrame([final])
    final_rev_1 = pd.concat([final_rev_1,data],ignore_index=True)

In [36]:
final_rev_1

,Img_name,Mask1,Mask2,Mask3,Mask4
0,A053_1614934171536.png,cksx20qpx000b266faqsm8afg,ckt2stelv005i266f5key9lsm,None,None
1,A053_1614934709068.png,cksx264ju000h266f2yccx5aj,ckt2stue7005k266fykrklgfc,None,None
2,A065_1615971661643.png,cksx2fvs3000j266fwval6rg0,ckt2sv4b3005m266fx4mpg0r1,None,None
3,A065_1615971902439.png,cksx2jaww000l266fxtegukgp,None,None,None
4,A087_1617961842943.png,cksx2rxm2000n266f2cuxdv4b,ckt2u956d005o266falanhu1z,None,None
...,...,...,...,...,...
382,M641_1625647191410.png,cku8fxw8t0009266hh2cz9jt0,None,None,None
383,M644_1625654505574.png,cku8g69pn000l266h7n4gca9m,None,None,None
384,M644_1625654542852.png,cku8g4das000i266hhutetlyq,None,None,None
385,M646_1625656611573.png,cku8gbnsc000r266hm7jomd4w,None,None,None


In [39]:
final_rev_1.nunique()

Img_name    354
Mask1       354
Mask2        33
Mask3         0
Mask4         0
dtype: int64

In [37]:
imgs = rev_2['External ID']
final_rev_2 = pd.DataFrame()
for item in rev_2['External ID']:
    final = {}
    masks = [None,None,None,None]
    values = rev_2[rev_2['External ID']==item]['Mask_ID']
    IDs = list(values)
    for num in range(len(IDs)):
        masks[num] = IDs[num]
    final['Img_name'] = item
    final['Mask1'] = masks[0]
    final['Mask2'] = masks[1]
    final['Mask3'] = masks[2]
    final['Mask4'] = masks[3]
    data = pd.DataFrame([final])
    final_rev_2 = pd.concat([final_rev_2,data],ignore_index=True)

In [41]:
final_rev_2.nunique()

Img_name    448
Mask1       448
Mask2         0
Mask3         0
Mask4         0
dtype: int64

In [38]:
final_rev_2

,Img_name,Mask1,Mask2,Mask3,Mask4
0,M071_1565687060188.png,cksxex9ru00003b6ar7bvgcoc,None,None,None
1,M075_1565781393764.png,cksxf4g4j000e3b6ajua625wu,None,None,None
2,M075_1565781696144.png,cksxfbgrq000o3b6agk3ju13o,None,None,None
3,M075_1565781731483.png,cksxfdkv3000s3b6a8038ihhp,None,None,None
4,M086_1566802065084.png,cksxfsqzd000w3b6atn3zku3u,None,None,None
...,...,...,...,...,...
443,U395_1628149453778.png,cktk39t5b000d3a6b1sijz3rj,None,None,None
444,U410_1629112545914.png,cktk410yd00003a6bw53alfny,None,None,None
445,U411_1629115879773.png,cktk41rey00023a6bv1e7jfm1,None,None,None
446,U412_1629195478080.png,cktk437px00083a6bsjhhpk6c,None,None,None


In [42]:
final_rev_1 = final_rev_1.drop(['Mask3','Mask4'],axis=1)
pd.DataFrame.to_csv(final_rev_1,'final_reviewer_1.csv',index=False)

In [43]:
final_rev_2 = final_rev_2.drop(['Mask2','Mask3','Mask4'],axis=1)
pd.DataFrame.to_csv(final_rev_2,'final_reviewer_2.csv',index=False)